In [3]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
import numpy as np
from sqlalchemy import create_engine

db_name = "timescale"
db_user = "admin"
db_password = "TimescalePassword"
db_host = "localhost"
db_port = "5433"
database_uri = f"postgresql://{db_user}:{db_password}@{db_host}:{db_port}/{db_name}"
engine = create_engine(database_uri)
df2 = pd.read_sql("SELECT * FROM filteredsensordatagrid", engine)

In [4]:
df2 = pd.read_sql("SELECT kesto, tunti, matka FROM filteredsensordatagrid", engine)
df2["kesto"] = pd.to_timedelta(df2["kesto"]).dt.total_seconds()
df2 = df2.dropna()
X = df2.drop("matka", axis=1)
y = df2["matka"]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
model = RandomForestRegressor(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

y_pred = model.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)

new_data = pd.DataFrame([[3600, 12]], columns=["kesto", "tunti"])
predicted_length = model.predict(new_data)
print(f"route length prediction: {predicted_length[0]} units")

route length prediction: 238.43 units
